# HyLoRADA: Hybrid Low-Rank Adaptation with Position-Adaptive Landmarks

**Validated Configuration**: rsLoRA + Position Bias + Position-Adaptive Landmarks  
**Improvement**: +18.37% PPL reduction (69.00 → 56.33) on WikiText-2  
**Parameters**: ~3.54M trainable (2.9% of GPT-2)

## Quick Start for Kaggle

1. Set accelerator to **GPU** (Settings > Accelerator > GPU)
2. Run cells in order
3. For comprehensive ablation, run `test_ablation_proper.py`

In [ ]:
# Setup
import warnings
warnings.filterwarnings('ignore')

# Disable tokenizers parallelism to avoid fork warnings
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
# Clone repo (Kaggle)
import os
if os.path.exists('hylorada'):
    %cd hylorada
    !git pull
    print("⚠️ Repo updated! Please RESTART KERNEL (Runtime > Restart Session) to reload modules")
else:
    !git clone https://github.com/SadiaTabassum1216/hylorada.git
    %cd hylorada

In [ ]:
# Install dependencies
!pip install -q transformers datasets accelerate tqdm

In [ ]:
# Check GPU
import torch
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 1. Quick Demo - HyLoRADA Components

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from hylorada import HyLoRADAConfig, HyLoRADAModel, LandmarkLoRA

# Test Position-Adaptive LandmarkLoRA
lm = LandmarkLoRA(
    hidden_size=768, 
    num_landmarks=8,
    max_positions=32768,
    num_buckets=32
)
x = torch.randn(1, 64, 768)
y = lm(x)
print(f"Position-Adaptive LandmarkLoRA: {x.shape} -> {y.shape}")
print(f"Landmark params: {sum(p.numel() for p in lm.parameters()):,}")
print(f"Architecture: {lm}")

In [ ]:
# Load base model (GPT-2)
model_name = "openai-community/gpt2"
base_model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ✅ VALIDATED CONFIGURATION (Recommended)
config = HyLoRADAConfig(
    lora_rank=16,                 # Validated rank
    lora_alpha=16.0,
    use_dora_magnitude=False,     # ✅ Disabled (causes -5.62% degradation)
    landmark_enabled=True,        # ✅ Enabled (+18.37% improvement, best result)
    num_landmarks=8,
    num_position_buckets=32,      # Position bucketing for landmarks
    position_bias_enabled=True,   # ✅ Enabled (+2.11% improvement, 64 params)
    # Long Context Features (Optional - not yet validated)
    s2_attn_enabled=False,        # Set True for S²-Attn (shifted sparse)
    s2_group_size=2048,
    rope_scaling_type=None,       # Set "linear" for RoPE scaling
)

print("✅ Validated Configuration:")
print(f"  - rsLoRA: +17.07% improvement")
print(f"  - Position Bias: +2.11% (64 params)")
print(f"  - Position-Adaptive Landmarks: +18.37% total (12.5K params)")
print(f"  - DoRA: Disabled (causes degradation)")
print(f"\nExpected: +18.37% improvement (69.00 → 56.33 PPL)")
print("\nComponents:", config.get_component_status())

In [ ]:
# Apply HyLoRADA to base model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move base model to device first
base_model = base_model.to(device)

# Apply HyLoRADA wrapper
model = HyLoRADAModel(base_model, config)

# Move entire model to device
model = model.to(device)

model.print_trainable_params()

## 2. Validation - Ablation Study

Run the comprehensive ablation test to validate all components:

In [ ]:
!python test_ablation_proper.py

## 4. Training Example (Optional)

Train HyLoRADA on a small dataset:

In [ ]:
# Training example (requires GPU)
from hylorada.trainer import HyLoRADATrainer, TrainingConfig, create_long_context_dataloader
from datasets import load_dataset

# Load training data
train_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
train_texts = [text.strip() for text in train_dataset["text"] 
               if text.strip() and len(text.strip()) > 100][:500]

# Create dataloader
train_dataloader = create_long_context_dataloader(
    dataset=train_texts,
    tokenizer=tokenizer,
    max_length=512,
    batch_size=4,
)

# Training config
train_config = TrainingConfig(
    num_epochs=1,
    learning_rate=1e-4,
    per_device_batch_size=4,
    gradient_accumulation_steps=4,
    mixed_precision="bf16",
    output_dir="./output/hylorada_run",
    logging_steps=10,
)

# Create trainer
trainer = HyLoRADATrainer(
    model=model,
    train_dataloader=train_dataloader,
    config=train_config,
)

# Train
print("🚀 Starting training...")
trainer.train()
print("\n✅ Training complete! Model saved to ./output/hylorada_run")